In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                     labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
#     aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
    aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.6974 eloss:2.4258 aloss2:1.2614 exploreP:0.9983
Episode:1 meanR:25.0000 R:33.0000 rate:0.0660 aloss:0.6975 eloss:2.4123 aloss2:1.2692 exploreP:0.9951
Episode:2 meanR:20.6667 R:12.0000 rate:0.0240 aloss:0.6981 eloss:2.3739 aloss2:1.2917 exploreP:0.9939
Episode:3 meanR:25.0000 R:38.0000 rate:0.0760 aloss:0.6965 eloss:2.3791 aloss2:1.2855 exploreP:0.9901
Episode:4 meanR:24.0000 R:20.0000 rate:0.0400 aloss:0.6959 eloss:2.3771 aloss2:1.2848 exploreP:0.9882
Episode:5 meanR:26.5000 R:39.0000 rate:0.0780 aloss:0.6961 eloss:2.2870 aloss2:1.3388 exploreP:0.9844
Episode:6 meanR:26.8571 R:29.0000 rate:0.0580 aloss:0.6986 eloss:2.2666 aloss2:1.3520 exploreP:0.9816
Episode:7 meanR:25.2500 R:14.0000 rate:0.0280 aloss:0.6966 eloss:2.2207 aloss2:1.3808 exploreP:0.9802
Episode:8 meanR:28.6667 R:56.0000 rate:0.1120 aloss:0.6957 eloss:2.2042 aloss2:1.3880 exploreP:0.9748
Episode:9 meanR:28.3000 R:25.0000 rate:0.0500 aloss:0.6940 eloss:2.1902 aloss2:1.3

Episode:80 meanR:23.3951 R:38.0000 rate:0.0760 aloss:0.6871 eloss:0.8916 aloss2:2.4589 exploreP:0.8291
Episode:81 meanR:23.5854 R:39.0000 rate:0.0780 aloss:0.6859 eloss:0.8764 aloss2:2.4786 exploreP:0.8259
Episode:82 meanR:23.4337 R:11.0000 rate:0.0220 aloss:0.6869 eloss:0.8499 aloss2:2.4958 exploreP:0.8250
Episode:83 meanR:23.3333 R:15.0000 rate:0.0300 aloss:0.6845 eloss:0.8613 aloss2:2.4959 exploreP:0.8238
Episode:84 meanR:23.2000 R:12.0000 rate:0.0240 aloss:0.6863 eloss:0.8598 aloss2:2.4939 exploreP:0.8228
Episode:85 meanR:23.3837 R:39.0000 rate:0.0780 aloss:0.6860 eloss:0.8523 aloss2:2.5095 exploreP:0.8197
Episode:86 meanR:23.3678 R:22.0000 rate:0.0440 aloss:0.6841 eloss:0.8194 aloss2:2.5360 exploreP:0.8179
Episode:87 meanR:23.9773 R:77.0000 rate:0.1540 aloss:0.6849 eloss:0.7799 aloss2:2.5717 exploreP:0.8117
Episode:88 meanR:24.0787 R:33.0000 rate:0.0660 aloss:0.6846 eloss:0.7525 aloss2:2.6027 exploreP:0.8090
Episode:89 meanR:24.2778 R:42.0000 rate:0.0840 aloss:0.6839 eloss:0.7204 

Episode:159 meanR:39.0300 R:49.0000 rate:0.0980 aloss:0.7056 eloss:-1.3592 aloss2:7.5285 exploreP:0.5913
Episode:160 meanR:39.3300 R:44.0000 rate:0.0880 aloss:0.6923 eloss:-1.4536 aloss2:7.3855 exploreP:0.5888
Episode:161 meanR:39.9000 R:72.0000 rate:0.1440 aloss:0.7073 eloss:-1.4500 aloss2:7.6169 exploreP:0.5846
Episode:162 meanR:40.1400 R:33.0000 rate:0.0660 aloss:0.7157 eloss:-0.9982 aloss2:7.8249 exploreP:0.5827
Episode:163 meanR:40.8000 R:101.0000 rate:0.2020 aloss:0.7007 eloss:-1.6060 aloss2:7.8470 exploreP:0.5770
Episode:164 meanR:41.0300 R:36.0000 rate:0.0720 aloss:0.7048 eloss:-1.5684 aloss2:8.0949 exploreP:0.5749
Episode:165 meanR:41.6300 R:101.0000 rate:0.2020 aloss:0.7092 eloss:-1.5513 aloss2:8.2396 exploreP:0.5693
Episode:166 meanR:43.2600 R:184.0000 rate:0.3680 aloss:0.7246 eloss:-1.7077 aloss2:8.7192 exploreP:0.5591
Episode:167 meanR:44.1600 R:107.0000 rate:0.2140 aloss:0.6988 eloss:-1.8822 aloss2:9.0807 exploreP:0.5532
Episode:168 meanR:44.4900 R:50.0000 rate:0.1000 alo

Episode:237 meanR:99.0800 R:86.0000 rate:0.1720 aloss:0.7571 eloss:-6.6049 aloss2:35.1252 exploreP:0.2547
Episode:238 meanR:99.8900 R:146.0000 rate:0.2920 aloss:0.7511 eloss:-6.6179 aloss2:35.1326 exploreP:0.2511
Episode:239 meanR:99.9000 R:69.0000 rate:0.1380 aloss:0.7321 eloss:-7.0181 aloss2:35.8613 exploreP:0.2495
Episode:240 meanR:100.4100 R:61.0000 rate:0.1220 aloss:0.7306 eloss:-7.2375 aloss2:36.0099 exploreP:0.2480
Episode:241 meanR:100.4900 R:65.0000 rate:0.1300 aloss:0.7698 eloss:-4.3336 aloss2:35.2181 exploreP:0.2465
Episode:242 meanR:100.6100 R:53.0000 rate:0.1060 aloss:0.7487 eloss:-7.3335 aloss2:35.2778 exploreP:0.2452
Episode:243 meanR:102.3600 R:192.0000 rate:0.3840 aloss:0.7430 eloss:-7.0386 aloss2:35.8130 exploreP:0.2407
Episode:244 meanR:102.7700 R:91.0000 rate:0.1820 aloss:0.7560 eloss:-3.0940 aloss2:36.2408 exploreP:0.2387
Episode:245 meanR:103.3900 R:81.0000 rate:0.1620 aloss:0.7469 eloss:-7.0818 aloss2:36.0850 exploreP:0.2368
Episode:246 meanR:103.9800 R:110.0000 

Episode:314 meanR:105.5800 R:336.0000 rate:0.6720 aloss:0.7505 eloss:-7.4305 aloss2:42.2280 exploreP:0.1146
Episode:315 meanR:106.0500 R:92.0000 rate:0.1840 aloss:0.7753 eloss:-5.8010 aloss2:42.5093 exploreP:0.1136
Episode:316 meanR:106.1900 R:83.0000 rate:0.1660 aloss:0.7173 eloss:-8.2832 aloss2:42.7097 exploreP:0.1128
Episode:317 meanR:107.2800 R:162.0000 rate:0.3240 aloss:0.7236 eloss:-7.7492 aloss2:42.8495 exploreP:0.1111
Episode:318 meanR:107.5100 R:105.0000 rate:0.2100 aloss:0.7132 eloss:-6.1433 aloss2:42.9677 exploreP:0.1100
Episode:319 meanR:108.2100 R:108.0000 rate:0.2160 aloss:0.7488 eloss:-4.7751 aloss2:42.8600 exploreP:0.1090
Episode:320 meanR:108.6700 R:132.0000 rate:0.2640 aloss:0.7354 eloss:-7.0182 aloss2:43.1192 exploreP:0.1077
Episode:321 meanR:109.1400 R:182.0000 rate:0.3640 aloss:0.7441 eloss:-7.3973 aloss2:42.9946 exploreP:0.1059
Episode:322 meanR:110.7000 R:253.0000 rate:0.5060 aloss:0.7210 eloss:-6.7283 aloss2:42.8512 exploreP:0.1035
Episode:323 meanR:109.9500 R:1

...

Episode:1855 meanR:180.1800 R:313.0000 rate:0.6260 aloss:0.4242 eloss:-0.5139 aloss2:27.4145 exploreP:0.0100
Episode:1856 meanR:185.0800 R:500.0000 rate:1.0000 aloss:0.4231 eloss:-0.3247 aloss2:27.7051 exploreP:0.0100
Episode:1857 meanR:187.8600 R:500.0000 rate:1.0000 aloss:0.4309 eloss:-0.2845 aloss2:27.8538 exploreP:0.0100
Episode:1858 meanR:187.7300 R:11.0000 rate:0.0220 aloss:0.4333 eloss:-0.1025 aloss2:27.9662 exploreP:0.0100
Episode:1859 meanR:187.7300 R:10.0000 rate:0.0200 aloss:0.4310 eloss:-1.4912 aloss2:27.7168 exploreP:0.0100
Episode:1860 meanR:192.6300 R:500.0000 rate:1.0000 aloss:0.4257 eloss:-0.4366 aloss2:27.9563 exploreP:0.0100
Episode:1861 meanR:194.1100 R:158.0000 rate:0.3160 aloss:0.4254 eloss:-0.3394 aloss2:27.9792 exploreP:0.0100
Episode:1862 meanR:193.7900 R:10.0000 rate:0.0200 aloss:0.4561 eloss:1.3044 aloss2:28.2123 exploreP:0.0100
Episode:1863 meanR:194.8300 R:126.0000 rate:0.2520 aloss:0.4357 eloss:-0.6429 aloss2:28.1642 exploreP:0.0100
Episode:1864 meanR:194.

Episode:1931 meanR:328.5900 R:500.0000 rate:1.0000 aloss:0.4312 eloss:-1.1710 aloss2:35.4959 exploreP:0.0100
Episode:1932 meanR:333.4900 R:500.0000 rate:1.0000 aloss:0.4305 eloss:-0.7832 aloss2:35.9247 exploreP:0.0100
Episode:1933 meanR:333.4800 R:10.0000 rate:0.0200 aloss:0.4292 eloss:0.0034 aloss2:35.9658 exploreP:0.0100
Episode:1934 meanR:336.9100 R:443.0000 rate:0.8860 aloss:0.4326 eloss:-0.4713 aloss2:35.5308 exploreP:0.0100
Episode:1935 meanR:341.5500 R:500.0000 rate:1.0000 aloss:0.4239 eloss:-0.6770 aloss2:35.6732 exploreP:0.0100
Episode:1936 meanR:341.5500 R:500.0000 rate:1.0000 aloss:0.4384 eloss:-0.9292 aloss2:35.9405 exploreP:0.0100
Episode:1937 meanR:341.5500 R:500.0000 rate:1.0000 aloss:0.4276 eloss:-0.9228 aloss2:36.1311 exploreP:0.0100
Episode:1938 meanR:346.4400 R:500.0000 rate:1.0000 aloss:0.4317 eloss:-0.9108 aloss2:36.2395 exploreP:0.0100
Episode:1939 meanR:348.6500 R:437.0000 rate:0.8740 aloss:0.4241 eloss:-1.0728 aloss2:36.7700 exploreP:0.0100
Episode:1940 meanR:34

Episode:2007 meanR:424.3400 R:500.0000 rate:1.0000 aloss:0.4099 eloss:-0.1499 aloss2:37.6187 exploreP:0.0100
Episode:2008 meanR:429.2300 R:500.0000 rate:1.0000 aloss:0.4093 eloss:-0.1062 aloss2:37.3669 exploreP:0.0100
Episode:2009 meanR:429.2300 R:500.0000 rate:1.0000 aloss:0.4171 eloss:-0.1531 aloss2:37.1162 exploreP:0.0100
Episode:2010 meanR:426.2800 R:10.0000 rate:0.0200 aloss:0.4339 eloss:-1.6468 aloss2:37.2374 exploreP:0.0100
Episode:2011 meanR:426.2800 R:500.0000 rate:1.0000 aloss:0.4095 eloss:-0.2029 aloss2:37.0734 exploreP:0.0100
Episode:2012 meanR:426.2800 R:500.0000 rate:1.0000 aloss:0.4117 eloss:-0.0064 aloss2:36.7910 exploreP:0.0100
Episode:2013 meanR:426.2800 R:500.0000 rate:1.0000 aloss:0.4107 eloss:-0.0628 aloss2:36.6036 exploreP:0.0100
Episode:2014 meanR:430.5100 R:500.0000 rate:1.0000 aloss:0.4119 eloss:0.0517 aloss2:36.1957 exploreP:0.0100
Episode:2015 meanR:430.3400 R:483.0000 rate:0.9660 aloss:0.4117 eloss:0.0303 aloss2:35.8669 exploreP:0.0100
Episode:2016 meanR:433

Episode:2083 meanR:361.8700 R:500.0000 rate:1.0000 aloss:0.3911 eloss:-0.1282 aloss2:21.0357 exploreP:0.0100
Episode:2084 meanR:356.9700 R:10.0000 rate:0.0200 aloss:0.4010 eloss:-0.1158 aloss2:20.8109 exploreP:0.0100
Episode:2085 meanR:355.5300 R:356.0000 rate:0.7120 aloss:0.3976 eloss:-0.0905 aloss2:20.7953 exploreP:0.0100
Episode:2086 meanR:355.5300 R:500.0000 rate:1.0000 aloss:0.3984 eloss:0.0126 aloss2:20.5836 exploreP:0.0100
Episode:2087 meanR:355.5300 R:500.0000 rate:1.0000 aloss:0.3981 eloss:0.0308 aloss2:20.0833 exploreP:0.0100
Episode:2088 meanR:356.7900 R:136.0000 rate:0.2720 aloss:0.4028 eloss:0.0537 aloss2:19.8468 exploreP:0.0100
Episode:2089 meanR:356.7900 R:500.0000 rate:1.0000 aloss:0.3972 eloss:0.0114 aloss2:19.5559 exploreP:0.0100
Episode:2090 meanR:356.7900 R:500.0000 rate:1.0000 aloss:0.3978 eloss:0.0080 aloss2:19.1726 exploreP:0.0100
Episode:2091 meanR:356.7900 R:500.0000 rate:1.0000 aloss:0.3940 eloss:-0.0053 aloss2:18.7100 exploreP:0.0100
Episode:2092 meanR:356.79

Episode:2159 meanR:235.9500 R:11.0000 rate:0.0220 aloss:0.3992 eloss:-0.2572 aloss2:12.6438 exploreP:0.0100
Episode:2160 meanR:233.9600 R:250.0000 rate:0.5000 aloss:0.3991 eloss:-0.2304 aloss2:12.6512 exploreP:0.0100
Episode:2161 meanR:231.9400 R:191.0000 rate:0.3820 aloss:0.4009 eloss:-0.2164 aloss2:12.6275 exploreP:0.0100
Episode:2162 meanR:232.9400 R:500.0000 rate:1.0000 aloss:0.3977 eloss:-0.2515 aloss2:12.6626 exploreP:0.0100
Episode:2163 meanR:229.6300 R:109.0000 rate:0.2180 aloss:0.3972 eloss:-0.2973 aloss2:12.6115 exploreP:0.0100


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
